# 🤖 Pranav's Personal AI Assistant - Fine-tuning on Google Colab

**Train an AI model on your personal profile, projects, and knowledge!**

This notebook trains a **Llama 3.2 3B** model on Pranav's profile information including:
- FTC/FRC robotics projects (Evergreen Dragons, Team 2854 Prototypes)
- DIY projects (Robotic Hand, Sim Racing Wheel, Robotic Arm)
- Technical preferences and skills
- General knowledge capabilities

**Hardware Requirements:** Runs comfortably on Google Colab Free (T4 GPU: 8GB VRAM, 36GB RAM)

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!

<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://unsloth.ai/docs/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>
</div>

## 📋 Table of Contents
1. [Installation](#Installation)
2. [Load Model](#Model)
3. [Upload Your Dataset](#Dataset)
4. [Train](#Train)
5. [Test the Model](#Inference)
6. [Save & Export](#Save)

<a name="Installation"></a>
## 🔧 Installation

Install Unsloth and required dependencies. This takes about 1-2 minutes.

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth  # Do this in local & cloud setups
else:
    import torch; v = re.match(r'[\d]{1,}\.[\d]{1,}', str(torch.__version__)).group(0)
    xformers = 'xformers==' + {'2.10':'0.0.34','2.9':'0.0.33.post1','2.8':'0.0.32.post2'}.get(v, "0.0.34")
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth_zoo bitsandbytes accelerate {xformers} peft trl triton unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

<a name="Model"></a>
## 🧠 Load Llama 3.2 3B Model

We use **Llama 3.2 3B Instruct** - perfect for 8GB VRAM! It's:
- Fast and efficient
- Great for conversational AI
- Supports 4-bit quantization for low memory usage
- Strong general knowledge + ability to learn your specific information

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Context window size
dtype = None  # Auto-detect: Float16 for T4, Bfloat16 for newer GPUs
load_in_4bit = True  # Use 4-bit quantization to fit in 8GB VRAM

print("🚀 Loading Llama 3.2 3B Instruct...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("✅ Model loaded successfully!")

## 🎯 Add LoRA Adapters

LoRA (Low-Rank Adaptation) lets us fine-tune efficiently by only updating 1-10% of parameters.
This saves memory and training time while maintaining quality!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,  # LoRA rank - higher = more capacity (using 32 for good quality)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,  # 0 is optimized
    bias = "none",
    use_gradient_checkpointing = "unsloth",  # Saves 30% VRAM!
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("✅ LoRA adapters added!")

<a name="Dataset"></a>
## 📂 Upload Your Dataset

**Option 1: Upload the dataset file**

Run the cell below and upload `pranav_profile_qa.jsonl` when prompted.

In [ ]:
from google.colab import files
import os

print("📤 Upload your pranav_profile_qa.jsonl file:")
uploaded = files.upload()

# Save to a known location
dataset_path = "pranav_profile_qa.jsonl"
if "pranav_profile_qa.jsonl" in uploaded:
    print(f"✅ Dataset uploaded successfully!")
    print(f"   File size: {len(uploaded['pranav_profile_qa.jsonl']) / 1024:.2f} KB")
else:
    print("⚠️  Please upload pranav_profile_qa.jsonl")

**Option 2: Download from GitHub**

If you have the dataset in a GitHub repo, uncomment and run:

In [ ]:
# !wget https://raw.githubusercontent.com/unslothai/notebooks/main/data/pranav_profile_qa.jsonl

## 📊 Load and Prepare Dataset

We'll load the JSONL dataset and convert it to Llama 3's chat format.

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

# Set up chat template for Llama 3.2
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",  # Llama 3.2 uses 3.1 format
)

# Load the dataset
print("📖 Loading dataset...")
dataset = load_dataset("json", data_files=dataset_path, split="train")

print(f"✅ Loaded {len(dataset)} training examples")
print(f"\nExample fields: {dataset.column_names}")
print(f"\nFirst example:")
print(dataset[0])

In [ ]:
# Format function to convert to chat format
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]

    texts = []
    for instruction, user_input, output in zip(instructions, inputs, outputs):
        # Create conversation with system instruction
        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": user_input},
            {"role": "assistant", "content": output}
        ]

        # Apply chat template
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)

    return {"text": texts}

# Apply formatting
print("🔄 Formatting dataset for training...")
dataset = dataset.map(formatting_prompts_func, batched=True)

print("✅ Dataset formatted!")
print(f"\nFormatted example (first 500 chars):")
print(dataset[0]["text"][:500] + "...")

<a name="Train"></a>
## 🏋️ Train the Model

Now let's fine-tune! This will take approximately 15-30 minutes depending on dataset size.

**Training Settings:**
- Batch size: 2 (fits in 8GB VRAM)
- Gradient accumulation: 4 steps (effective batch size of 8)
- Steps: 300 (increase for more training)
- Learning rate: 2e-4 (standard for LoRA)

In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False,  # Can enable for speed with short sequences
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 300,  # Increase to 600+ for stronger learning
        learning_rate = 2e-4,
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

print("✅ Trainer configured!")

### Train Only on Assistant Responses

We'll mask the user inputs so the model only learns to generate assistant responses.

In [ ]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

print("✅ Training will focus on assistant responses only!")

In [ ]:
# Check memory before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"🎮 GPU: {gpu_stats.name}")
print(f"💾 Max memory: {max_memory} GB")
print(f"📊 Memory reserved: {start_gpu_memory} GB")

In [ ]:
# Start training!
print("🚀 Starting training...")
print("⏰ This will take approximately 15-30 minutes...")
print("=" * 50)

trainer_stats = trainer.train()

print("=" * 50)
print("✅ Training complete!")

In [ ]:
# Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

print("📊 Training Statistics:")
print(f"⏱️  Time: {round(trainer_stats.metrics['train_runtime']/60, 2)} minutes")
print(f"💾 Peak memory: {used_memory} GB ({used_percentage}% of {max_memory} GB)")
print(f"🎯 Memory for training: {used_memory_for_lora} GB ({lora_percentage}%)")

<a name="Inference"></a>
## 🎯 Test Your Model!

Let's test the trained model with questions about Pranav's profile and projects.

In [ ]:
from transformers import TextStreamer

# Enable fast inference
FastLanguageModel.for_inference(model)

print("✅ Model ready for inference!")
print("🎯 Let's test with some questions about Pranav's profile...\n")

### Test 1: FTC Team Information

In [ ]:
messages = [
    {"role": "user", "content": "What FTC team is Pranav on and what are his leadership goals?"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
print("Question: What FTC team is Pranav on and what are his leadership goals?\n")
print("Answer: ", end="")
_ = model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    max_new_tokens = 128,
    use_cache = True,
    temperature = 0.7,
    top_p = 0.9
)
print("\n" + "="*70 + "\n")

### Test 2: DIY Projects

In [ ]:
messages = [
    {"role": "user", "content": "Tell me about Pranav's sim racing steering wheel build."},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
print("Question: Tell me about Pranav's sim racing steering wheel build.\n")
print("Answer: ", end="")
_ = model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    max_new_tokens = 150,
    use_cache = True,
    temperature = 0.7,
    top_p = 0.9
)
print("\n" + "="*70 + "\n")

### Test 3: Technical Preferences

In [ ]:
messages = [
    {"role": "user", "content": "What CAD software does Pranav use and what's his coding preference?"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
print("Question: What CAD software does Pranav use and what's his coding preference?\n")
print("Answer: ", end="")
_ = model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    max_new_tokens = 128,
    use_cache = True,
    temperature = 0.7,
    top_p = 0.9
)
print("\n" + "="*70 + "\n")

### Test 4: General Knowledge (Not in Profile)

In [ ]:
messages = [
    {"role": "user", "content": "What is the capital of France?"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
print("Question: What is the capital of France?\n")
print("Answer: ", end="")
_ = model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    max_new_tokens = 100,
    use_cache = True,
    temperature = 0.7,
    top_p = 0.9
)
print("\n" + "="*70 + "\n")

### 🎮 Try Your Own Questions!

Modify the question below to test any topic:

In [ ]:
# ✏️ Edit the question here:
custom_question = "What motors does Pranav use in his sim racing wheel?"

messages = [
    {"role": "user", "content": custom_question},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
print(f"Question: {custom_question}\n")
print("Answer: ", end="")
_ = model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    max_new_tokens = 150,
    use_cache = True,
    temperature = 0.7,
    top_p = 0.9
)
print("\n")

<a name="Save"></a>
## 💾 Save Your Model

Now let's save the fine-tuned model! You have several options:

### Option 1: Save LoRA Adapters (Small, ~200MB)

This saves only the trained adapter weights. You'll need the base model + adapters to run it.

In [ ]:
print("💾 Saving LoRA adapters...")

model.save_pretrained("pranav_assistant_lora")
tokenizer.save_pretrained("pranav_assistant_lora")

print("✅ LoRA adapters saved to 'pranav_assistant_lora' folder!")
print("📦 You can download this folder from the Colab file browser (left sidebar)")

### Option 2: Upload to Hugging Face Hub (Recommended!)

Upload to Hugging Face so you can use it anywhere. Get your token at: https://huggingface.co/settings/tokens

In [ ]:
# Uncomment and fill in your details:
# HF_USERNAME = "your-username"  # Your Hugging Face username
# HF_TOKEN = "hf_..."  # Your Hugging Face token

# model.push_to_hub(f"{HF_USERNAME}/pranav-assistant-3b-lora", token=HF_TOKEN)
# tokenizer.push_to_hub(f"{HF_USERNAME}/pranav-assistant-3b-lora", token=HF_TOKEN)

# print(f"✅ Model uploaded to: https://huggingface.co/{HF_USERNAME}/pranav-assistant-3b-lora")

### Option 3: Save Merged 16-bit Model (Larger, ~6GB)

This merges LoRA weights into the base model for standalone use.

In [ ]:
# Uncomment to save merged model:
# print("💾 Saving merged 16-bit model (this may take a few minutes)...")
# model.save_pretrained_merged("pranav_assistant_merged_16bit", tokenizer, save_method="merged_16bit")
# print("✅ Merged model saved!")

### Option 4: Save as GGUF for llama.cpp / Ollama

GGUF format works with llama.cpp, Ollama, LM Studio, and other tools.

In [ ]:
# Uncomment to save as GGUF (Q8_0 quantization):
# print("💾 Saving GGUF model...")
# model.save_pretrained_gguf("pranav_assistant", tokenizer, quantization_method="q8_0")
# print("✅ GGUF model saved!")

# For better compression, try q4_k_m:
# model.save_pretrained_gguf("pranav_assistant", tokenizer, quantization_method="q4_k_m")

### 📥 Download Your Model

To download from Colab:
1. Click the folder icon 📁 in the left sidebar
2. Find your saved model folder
3. Right-click → Download

Or use this code to zip and download:

In [ ]:
# Uncomment to zip and download:
# !zip -r pranav_assistant_lora.zip pranav_assistant_lora/
# from google.colab import files
# files.download("pranav_assistant_lora.zip")

## 🔄 Reload Your Model Later

To use your saved model in a new session:

In [ ]:
# Uncomment to reload from saved LoRA:
# from unsloth import FastLanguageModel
#
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "pranav_assistant_lora",  # Your saved folder
#     max_seq_length = 2048,
#     dtype = None,
#     load_in_4bit = True,
# )
# FastLanguageModel.for_inference(model)
#
# # Now you can use it for inference!

## 🎉 Congratulations!

You've successfully trained a personalized AI assistant! Here's what you accomplished:

✅ Fine-tuned Llama 3.2 3B on your personal profile  
✅ Trained on 8GB VRAM (Google Colab Free)  
✅ Model learns your projects, preferences, and background  
✅ Maintains general knowledge capabilities  
✅ Saved for future use  

### 🚀 Next Steps:

1. **Increase Training**: Change `max_steps` to 600-1000 for stronger learning
2. **Add More Data**: Expand your dataset with more details and examples
3. **Deploy It**: Use on Ollama, LM Studio, or your own server
4. **Share It**: Upload to Hugging Face for others to try

### 📚 Resources:

- **Unsloth Docs**: https://unsloth.ai/docs/
- **Discord Community**: https://discord.gg/unsloth
- **GitHub**: https://github.com/unslothai/unsloth

### 💡 Tips for Better Results:

- More diverse examples = better generalization
- Include both factual Q&A and conversational examples
- Test regularly and add edge cases to your dataset
- For specialized knowledge, increase training steps

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>

  ⭐️ If this helped you, star <a href="https://github.com/unslothai/unsloth">Unsloth on GitHub</a>! ⭐️
</div>